# Tutorial: Zero-Shot Learning for Beverage Classification in Images

In this tutorial we are going to implement Zero-Shot Learning (ZSL) using two popular deep learning models: CLIP (Contrastive Language-Image Pre-training) and LLaVA (Large Language and Vision Assistant model). This notebook is available in two forms:

1.   [Online (Google Colab)](https://colab.research.google.com/github/ltu-capr/zsl-image-tutorial/blob/master/ZSL_for_image_beverage_classification.ipynb): For experimenting on Google's free platform without installing anything on your computer.
2.   [Offline (Jupyter Notebook)](https://github.com/ltu-capr/zsl-image-tutorial): For experimenting locally on your own computer. This takes some additional setup, but is the best option for working with sensitive data.

To run the code in a cell, either click inside it and press Ctrl + Enter, or click the 'play' button to the left of the cell.

*The ZSL models at the core of this notebook run much faster with graphics processing unit (GPU) acceleration. If you are in Google Colab, you can enable GPU acceleration in the settings by going to Runtime > Change runtime type > Hardware accelerator (select "GPU").*

## Example scenario: Beverage Classification in Images

In this scenario, we want to classify the type of beverage depicted in an image from a set of predefined categories, and thus, determine if an alcoholic beverage is present.

The candidate beverage types we aim to classify are:

*    Beer
*    Wine
*    Water
*    Coffee
*    Tea
*    No Beverage

To approach this, we will use two popular deep learning models that operate in different ways:

* CLIP (Contrastive Language-Image Pre-training)
* LLaVA (Large Language and Vision Assistant)


### Cell 0.0: Install software package requirements


*   Pandas is used to load and save data in CSV (comma separated value) format.
*   PyTorch and Transformers are used to run the models.
*   bitsandbytes and accelerate are used to quantise the LLaVA model, such that we can run it on machines with smaller amounts of available memory.
*   tqdm is used to show progress bars.
*   scikit-learn is used for metric computation.
*   matplotlib is used for visualising the confusion matrix and examples in the dataset.
*   numpy is used for efficient numerical computations.

In [ ]:
!pip install pandas torch transformers==4.46.0 bitsandbytes==0.45.0 accelerate==1.2.0 tqdm scikit-learn matplotlib numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 1.6 MB/s eta 0:00:00
Reason for being yanked: This version unfortunately does not work with 3.8 but we did not drop the support yet
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.3/336.3 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━ 361.4/664.8 MB 98.3 MB/s eta 0:00:04

### Cell 0.1: Import essential modules

In [ ]:
import os
from PIL import Image

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from sklearn.metrics import ConfusionMatrixDisplay, accuracy_score, precision_score, recall_score, f1_score
from tqdm.notebook import tqdm
from transformers import CLIPProcessor, CLIPModel, LlavaNextProcessor, LlavaNextForConditionalGeneration, BitsAndBytesConfig

### Cell 0.2 Define useful functions
Here we define some useful functions to help compute metrics we want to report for both models. The code here may look daunting, however it is not crucial to fully understand. Instead, you can run the code cell and move on to the next cell.

In [ ]:
def per_class_sensitivity_specificity(ground_truth_labels, predicted_labels, label_names):
    """Computes per-class sensitivity (recall) and specificity metrics.

    Metrics returned in order of label_names.
    """
    sensitivity_scores, specificity_scores = [], []
    for label_name in label_names:
        # Turn into binary problem. recall_score returns specificity then sensitivity
        specificity, sensitivity = recall_score(
            np.array(ground_truth_labels) == label_name, np.array(predicted_labels) == label_name,
            pos_label=True, average=None, zero_division=np.nan)
        sensitivity_scores.append(sensitivity)
        specificity_scores.append(specificity)
    return sensitivity_scores, specificity_scores


def generate_metric_report(ground_truth_labels, predicted_labels, label_names):
    """Computes and displays a report containing a collection of metrics.

    Per-class metrics include:
    - Precision
    - Recall (sensitivity)
    - Specificity
    - F1-Score (harmonic mean of precision and recall)

    Overall metrics include:
    - Accuracy
    - Unweighted Average Recall (UAR)
    """
    # Determine the length of the longest label name to format the table.
    len_longest_label = max(len(label_name) for label_name in label_names)

    # Compute per-class metrics.
    per_class_precision = precision_score(ground_truth_labels, predicted_labels, labels=label_names, average=None, zero_division=np.nan)
    per_class_recall, per_class_specificity = per_class_sensitivity_specificity(ground_truth_labels, predicted_labels, label_names=label_names)
    per_class_f1_score = f1_score(ground_truth_labels, predicted_labels, labels=label_names, average=None, zero_division=np.nan)

    # Display the per-class summary.
    print(f'{"":^{len_longest_label}s}    {"precision":>9s}    {"recall":>6s}    {"specificity":>11s}    {"f1-score":>9s}    {"support":>6s}\n')
    for label_idx, label_name in enumerate(label_names):
        print(f'{label_name:>{len_longest_label}s}    {per_class_precision[label_idx]:>9.2f}    {per_class_recall[label_idx]:>6.2f}    {per_class_specificity[label_idx]:>11.2f}    {per_class_f1_score[label_idx]:>9.2f}    {ground_truth_labels.count(label_name):>6d}')
    print('\n')

    # Compute overall metrics.
    accuracy = accuracy_score(ground_truth_labels, predicted_labels)
    uar = recall_score(ground_truth_labels, predicted_labels, labels=label_names, average='macro', zero_division=np.nan)

    # Display the overall metrics.
    print(f'{"Number of Examples":>35s}: {len(ground_truth_labels)}')
    print(f'{"Overall Accuracy":>35s}: {accuracy*100:.2f}%')
    print(f'{"Unweighted Average Recall (UAR)":>35s}: {uar:.2f}')

### Cell 0.3: Load the "beverage" dataset

To evaluate model performance across different beverage types and scenes, we have constructed a dataset consisting of the following beverages:

**Alcoholic Beverages**
*   Beer bottle
*   Beer cup
*   Red wine glass
*   White wine glass


**Non-Alcoholic Beverages**
*   Coffee cup
*   Coffee plunger
*   Tea cup
*   Water bottle
*   Water cup


Multiple images of each beverage have been captured across five distinct scenes, both indoor and outdoor, with the beverage positioned at three different distances from the camera: foreground, midground, and background. The image below shows examples of the images contained in the dataset:

<div>
<img src="https://github.com/ltu-capr/zsl-image-tutorial/blob/main/dataset_example.jpg?raw=1" width="70%"/>
</div>

In total, for each of the nine beverage types, images have been taken across five different scenes and at three focal distances, resulting in 15 images per beverage. This leads to a complete dataset of 135 images (9 beverages $\times$ 15 images each).

**Note for offline notebook version**: You may encounter an error when downloading the image dataset when running the below code cell. If so:
1. Manually download the dataset from [here](https://github.com/ltu-capr/zsl-image-tutorial/raw/refs/heads/main/Data/Images.zip).
2. Extract the downloaded ZIP folder and place the folder in the same location as this notebook. The name of the folder should be 'Images', and inside the folder should be 135 images.

In [ ]:
# Download and extract the zipped image dataset.
!wget -O images.zip 'https://github.com/ltu-capr/zsl-image-tutorial/raw/refs/heads/main/Data/Images.zip'
!unzip -o -q images.zip -d ./Images

# Load the CSV file containing the labels for each image.
# Here we are giving the URL for a sample file that we've made publicly
# available on the Internet.
labels_location = 'https://raw.githubusercontent.com/ltu-capr/zsl-image-tutorial/main/Data/images_labelled.csv'
beverage_dataframe = pd.read_csv(labels_location)

### Cell 0.4: Visualise the "beverage" dataset
Here we visualise a few of the examples from the dataset.

In [ ]:
# Define how many images to show.
rows, cols = 3, 6
num_images = rows * cols

# Create a 3x6 grid for displaying images.
fix, axes = plt.subplots(3, 6, figsize=(18, 12))
axes = axes.flatten()

for index, sample in beverage_dataframe[:num_images].iterrows():
    # Load the image and create a small thumbnail.
    im = Image.open('Images/' + sample['image_name'] + '.jpg')
    im.thumbnail((256, 256), Image.Resampling.LANCZOS)

    # Display the image, setting the title as the beverage type and focal location.
    axes[index].imshow(im)
    axes[index].set_title(f'{sample["beverage"]}/{sample["position"]}')

    # Hide the axis markers.
    axes[index].tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)

# Display the plot.
plt.show()

## Example 1: ZSL Classification of Beverages in Images Using CLIP

CLIP works by taking an image and a collection of descriptors, and generates an image-text similarity score for each image-descriptor pair. With this, we can provide CLIP an image along with a set of candidate labels, and generate a prediction by using the candidate label with the highest image-text similarity score.

### Cell 1.0: Initialise the CLIP model

Initialise the CLIP model for use in zero-shot image classification. It may take a while for the model to download.

In [ ]:
# Use the GPU if it's available.
clip_device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

# Load the pre-trained model.
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(clip_device)

# Load an object used to prepare images in the right way required by CLIP
# This will: resize, centrecrop, and normalise the images.
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

### Cell 1.1: Initialise classification labels

In order to perform classification, we must nominate candidate labels for the model to choose between. In this scenario we have 6x labels, however you can choose as many labels as you need.

It is possible to further engineer the text in these labels.

In [ ]:
candidate_labels = [
    'A photo containing beer',
    'A photo containing wine',
    'A photo containing coffee',
    'A photo containing tea',
    'A photo containing water',
    'A photo containing no drinks'
]

### Cell 1.2: Make model predictions

Here we generate predictions for all images in our dataset.

In this example we also implement *batching*, enabling the model to process multiple images simultaneously. This is especially useful for larger datasets, as it can reduce computational overhead and speed up predictions.

In [ ]:
# Set the batch size (how many images to process at once).
clip_batch_size = 10

# Generate prediction results for all images in our dataset.
all_results = []
for index in tqdm(range(0, len(beverage_dataframe), clip_batch_size)):
    # Extract the next set of data to process.
    next_data = beverage_dataframe.iloc[index:index + clip_batch_size]

    # Open the images.
    images = [Image.open('Images/' + sample['image_name'] + '.jpg') for _, sample in next_data.iterrows()]

    # Preprocess the data to the correct format.
    inputs = clip_processor(
        text=candidate_labels,
        images=images, return_tensors='pt', padding=True
    ).to(clip_device)

    # Get the model outputs.
    with torch.no_grad():
        outputs = clip_model(**inputs)

    # Get the logits per-image (the image-text similarity score to each candidate label).
    image_logits = outputs.logits_per_image.cpu()

    # Convert logits per-image into probabilities with a softmax, then convert to a list.
    image_probs = image_logits.softmax(dim=1)
    image_probs = list(torch.unbind(image_probs, dim=0))

    # Store the probabilities.
    all_results.extend(image_probs)

    # Visualise last result in batch as model is running.
    last_image = images[-1]
    last_image_name = next_data.iloc[-1]['image_name']
    last_image_probs = all_results[-1]

    # Determine the index of the highest probability.
    highest_idx = torch.argmax(last_image_probs)

    # Prepare the image for displaying
    thumb_im = last_image.copy()
    thumb_im.thumbnail((256, 256), Image.Resampling.LANCZOS)
    display(thumb_im)

    print(f'Most likely label for {last_image_name}: {candidate_labels[highest_idx]} (probability: {100 * last_image_probs[highest_idx].item():.1f}%)')
    print(f'Per-class probabilities:')
    for lbl_idx, label in enumerate(candidate_labels):
        print(f'\t{label}: {100 * last_image_probs[lbl_idx]:.1f}%')

### Cell 1.3 Save the model predictions

This code prepares an output CSV file containing model predictions which can be used for further analysis.

In [ ]:
def save_clip_predictions(file_name, dataframe, all_results, candidate_labels, actual_label_column=None):
    # Arrange the results in tabular form with neat columns.
    rows = []
    for result, (index, sample) in zip(all_results, dataframe.iterrows()):
        scores_as_percentages = [round(score * 100, 2) for score in result.tolist()]
        row = {'image_name': sample['image_name'], **dict(zip(candidate_labels, scores_as_percentages))}
        rows.append(row)
    results_df = pd.DataFrame(rows, columns=['image_name', *candidate_labels])
    results_df['predicted_label'] = results_df[candidate_labels].idxmax(axis=1)

    if actual_label_column is not None and actual_label_column in dataframe.columns:
        results_df['actual_label'] = 'A photo containing ' + dataframe[actual_label_column]

    # Save output to a CSV file.
    os.makedirs('Outputs', exist_ok=True)
    output_file_name = os.path.join('Outputs', file_name)
    results_df.to_csv(output_file_name, index=False)

    try:
        # If we are on Google Colab, download the results.
        from google.colab import files
        files.download(output_file_name)
    except ModuleNotFoundError:
        # If we are not on Google Colab, show the output file location.
        print('Output file saved:')
        print(os.path.abspath(output_file_name))


save_clip_predictions('beverage_classification_clip_predictions.csv',
                      beverage_dataframe, all_results, candidate_labels,
                      'beverage_type')

### Cell 1.4. Measure the accuracy of model predictions (optional)

Zero-shot learning does not require hand-annotated labels to generate predictions, but they can be used to validate the model's accuracy. Here we compare the model's outputs with hand-annotated (ground truth) labels. If you don't have hand-annotated labels for your data, skip this step.

#### Cell 1.4a. Metric computation
In the code cell below, we will generate a table that summarises the precision, recall, specificity and F1 score for each beverage type individually, in addition to other statistics summarising the overall model performance, such as the overall accuracy and unweighted average recall (UAR).

In the generated report, *support* refers to the number of ground truth labels belonging to that class.

In [ ]:
# Put the predictions and ground truth into the right format.
predicted_labels = [candidate_labels[torch.argmax(result)] for result in all_results]
ground_truth_labels = list('A photo containing ' + beverage_dataframe['beverage_type'])

# Produce the metrics report.
generate_metric_report(ground_truth_labels, predicted_labels, label_names=candidate_labels)

These statistics show that CLIP only achieves an overall accuracy of 34.07%. Though there is no baseline for accuracy, this is still a poor result, and suggests further engineering of the classification labels may be needed, or a different model may be more appropriate for this data.

#### Cell 1.4b. Confusion matrix

We can analyse the model's performance in a more detailed manner by visualising a confusion matrix. A confusion matrix shows how many dataset examples there are for each possible pair of true and predicted labels. Numbers which do not lie on the main diagonal of the matrix correspond to misclassifications. By inspecting the classification matrix, we can quickly observe specific classes that the model is performing poorly on. For example, in this case we can see the model is best at correctly identifying photos containing beer, wine, and coffee, however struggles with other beverage types, such as water.

In [ ]:
ConfusionMatrixDisplay.from_predictions(ground_truth_labels, predicted_labels, labels=candidate_labels, xticks_rotation='vertical', cmap='Blues', colorbar=False);

#### Cell 1.4c. Foreground/midground/background performance

We can take a closer look at what other factors might be causing this misclassification. For instance, we can look at the classification performance in foreground, midground, and background. In this case, we can see that the model is much better at making correct predictions when the beverage is in the foreground as opposed to the midground or background.

In [ ]:
for position in ('foreground', 'midground', 'background'):
    predicted_labels = []
    ground_truth_labels = []
    for result, (index, sample) in zip(all_results, beverage_dataframe.iterrows()):
        if sample['position'] == position:
            predicted_labels.append(candidate_labels[torch.argmax(result)])
            ground_truth_labels.append('A photo containing ' + sample['beverage_type'])

    accuracy = accuracy_score(ground_truth_labels, predicted_labels)
    print(f'Accuracy for {position}: {accuracy:.2%}', flush=True)

    ConfusionMatrixDisplay.from_predictions(ground_truth_labels, predicted_labels, labels=candidate_labels, xticks_rotation='vertical', cmap='Blues', colorbar=False);
    plt.show()

#### Cell 1.4d. Alcohol vs. not-alcohol evaluation

Finally, we can take a look at how well the model performed at correctly classifying whether an image contains an alcoholic beverage (beer or wine). If no beverage was detected, we classify this as no alcohol present.

In [ ]:
# Put the predictions and ground truth into the right format.
predicted_labels = [candidate_labels[torch.argmax(result)] for result in all_results]
predicted_labels = ['alcohol' if 'beer' in label or 'wine' in label else 'not alcohol' for label in predicted_labels]
ground_truth_labels = list(beverage_dataframe['alcohol_notalcohol'])

# Produce the metrics report.
generate_metric_report(ground_truth_labels, predicted_labels, label_names=['alcohol', 'not alcohol'])

# Generate the confusion matrix.
ConfusionMatrixDisplay.from_predictions(ground_truth_labels, predicted_labels, labels=['alcohol', 'not alcohol'], xticks_rotation='vertical', cmap='Blues', colorbar=False);

## Example 2 - ZSL Classification of Beverages in Images Using LLaVA

LLaVA is a large multimodal model that can accept both images and text as input. With LLaVA, we can provide an image along with a question, such as "What beverage is contained in the image?" or "Does the image contain an alcoholic beverage?". To make the answer suitable for classification, we can also ask the model to restrict its response to a set of candidate categories or a simple yes or no.

### Cell 2.0 Initialise the LLaVA model

Initialise the LLaVA model for use in zero-shot image classification. It will take a while for the model to download.

LLaVA is quite a large model, which by default might not be able to run on most machines due to memory constraints. To support most machines, we will use a quantised version of the model, that reduces the memory requirements at a small cost to accuracy.

This process requires the use of a GPU, meaning that a GPU is required to run the quantised version of LLaVA.

In [ ]:
# Must use the GPU when running a quantised LLaVA.
if not torch.cuda.is_available():
    raise RuntimeError('Must have a GPU available to run a quantised version of LLaVA.')
llava_device = 'cuda:0'

# Specify which version of LLaVa we will use.
model_id = 'llava-hf/llava-v1.6-mistral-7b-hf'

# Set up parameters relating to model quantisation.
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
)

# Load the pre-trained model.
llava_model = LlavaNextForConditionalGeneration.from_pretrained(
    model_id, quantization_config=quantization_config)

# Load an object used to prepare data in the right way required by LLaVA.
llava_processor = LlavaNextProcessor.from_pretrained(
    model_id, patch_size=llava_model.config.vision_config.patch_size,
    vision_feature_select_strategy=llava_model.config.vision_feature_select_strategy,
)
llava_model.generation_config.pad_token_id = llava_processor.tokenizer.eos_token_id

### Cell 2.1 Initialise prompt

In order to perform classification we must first create a prompt for the LLaVA model. Here, we will ask the model to identify the presence of any specific beverages (beer, wine, coffee, tea, water) or none in the image. This approach allows us to handle the task as as a multi-class classification problem, similar to what we did for CLIP.

We also limit the maximum number of tokens (words or word parts) that LLaVA can generate in its response. Here, we set it to a low value (10) as we only expect a single-word answer, though this choice is less critical as our prompt specifically instructs LLaVA to respond with only one word. Raising this limit would let LLaVA generate longer responses, which can be useful if a more detailed answer is needed.

In [ ]:
# Here we specify the prompt to give to LLaVA.
prompt = 'What type of beverage does this image contain? Pick the most relevant one from this list: beer, wine, coffee, tea, water, none. Answer in one word.'

# Here we specify how many tokens (words or word parts) can be returned by LLaVa. We restrict this to a small number.
max_new_tokens = 10

### Cell 2.2 Make model predictions

Here we generate predictions for all images in our dataset.

In this example we also implement *batching*, enabling the model to process multiple images simultaneously. This is especially useful for larger datasets, as it can reduce computational overhead and speed up predictions.

Depending on the GPU used, a smaller batch size may be required due to memory constraints.

In [ ]:
# Set the batch size (how many images to process at once).
llava_batch_size = 10

# Generate prediction results for all images in our dataset.
all_results = []
for index in tqdm(range(0, len(beverage_dataframe), llava_batch_size)):
    # Extract the next set of data to process.
    next_data = beverage_dataframe.iloc[index:index + llava_batch_size]

    # Open the images.
    images = [Image.open('Images/' + sample['image_name'] + '.jpg') for _, sample in next_data.iterrows()]

    # Create the formatted prompt for LLaVA.
    llava_prompt = f'[INST] <image>\n{prompt} [/INST]'

    # Prepare the inputs to pass to LLaVA.
    inputs = llava_processor(
        text=[llava_prompt] * len(images),
        images=images, return_tensors='pt'
    ).to(llava_device)

    # Get the model predictions.
    with torch.no_grad():
        outputs = llava_model.generate(**inputs, max_new_tokens=max_new_tokens)

    # Process all predictions.
    for output in outputs:
        # Get the generated text and extract just the response.
        generated_text = llava_processor.decode(output.cpu(), skip_special_tokens=True)
        response = generated_text.split(f'{prompt} [/INST]')[1].strip().lower()

        # Store the response.
        all_results.append(response)

    # Visualise last result in batch as model is running.
    last_image = images[-1]
    last_image_name = next_data.iloc[-1]['image_name']
    last_image_response = all_results[-1]

    # Prepare the image for displaying
    thumb_im = last_image.copy()
    thumb_im.thumbnail((256, 256), Image.Resampling.LANCZOS)
    display(thumb_im)

    print(f'Image: {last_image_name}')
    print(f'Prompt: {prompt}')
    print(f'Response: {last_image_response}')

### Cell 2.3 Save the model predictions

This code prepares an output CSV file containing model predictions which can be used for further analysis.

In [ ]:
def save_llava_predictions(file_name, dataframe, all_results, actual_label_column=None):
    # Arrange the results in tabular form with neat columns.
    results_df = pd.DataFrame([])
    results_df['image_name'] = dataframe['image_name']
    results_df['predicted_label'] = all_results

    if actual_label_column is not None and actual_label_column in dataframe.columns:
        results_df['actual_label'] = dataframe[actual_label_column]

    # Save output to a CSV file.
    os.makedirs('Outputs', exist_ok=True)
    output_file_name = os.path.join('Outputs', file_name)
    results_df.to_csv(output_file_name, index=False)

    try:
        # If we are on Google Colab, download the results.
        from google.colab import files
        files.download(output_file_name)
    except ModuleNotFoundError:
        # If we are not on Google Colab, show the output file location.
        print('Output file saved:')
        print(os.path.abspath(output_file_name))

save_llava_predictions('beverage_classification_llava_predictions.csv',
                       beverage_dataframe, all_results, 'beverage_type')

### Cell 2.4 Measure the accuracy of model predictions (optional)

Zero-shot learning does not require hand-annotated labels to generate predictions, but they can be used to validate the model's accuracy. Here we compare the model's outputs with hand-annotated (ground truth) labels. If you don't have hand-annotated labels for your data, skip this step.

#### Cell 2.4a. Metric computation

In the code cell below, we will generate a table that summarises the precision, recall, specificity and F1 score for each beverage type individually, in addition to other statistics summarising the overall model performance, such as the overall accuracy and unweighted average recall (UAR).

In the generated report, *support* refers to the number of ground truth labels belonging to that class.

In [ ]:
# Put the predictions and ground truth into the right format.
predicted_labels = all_results
ground_truth_labels = list(beverage_dataframe['beverage_type'])

# Produce the metrics report.
generate_metric_report(ground_truth_labels, predicted_labels, label_names=('beer', 'wine', 'coffee', 'tea', 'water', 'none'))

These results look much better than for CLIP, with an overall accuracy of 79.26%.

#### Cell 2.4b. Confusion matrix

We can analyse accuracy in a more detailed manner by visualising a confusion matrix. Here we can see the model does a really good job! In this case we can see the model often incorrectly classifies tea as coffee, however distinguishing between the two is very challenging!

In [ ]:
ConfusionMatrixDisplay.from_predictions(ground_truth_labels, predicted_labels, labels=('beer', 'wine', 'coffee', 'tea', 'water', 'none'), xticks_rotation='vertical', cmap='Blues', colorbar=False);

#### Cell 2.4c. Foreground/midground/background performance

We can further analyse the classification performance in foreground, midground, and background. Similarly to CLIP, the model performs better when the beverage is contained in the foreground, and worse when in the background.

In [ ]:
for position in ('foreground', 'midground', 'background'):
    predicted_labels = []
    ground_truth_labels = []
    for result, (index, sample) in zip(all_results, beverage_dataframe.iterrows()):
        if sample['position'] == position:
            predicted_labels.append(result)
            ground_truth_labels.append(sample['beverage_type'])

    accuracy = accuracy_score(ground_truth_labels, predicted_labels)
    print(f'Accuracy for {position}: {accuracy:.2%}', flush=True)

    ConfusionMatrixDisplay.from_predictions(ground_truth_labels, predicted_labels, labels=('beer', 'wine', 'coffee', 'tea', 'water', 'none'), xticks_rotation='vertical', cmap='Blues', colorbar=False);
    plt.show()

#### Cell 2.4d. Alcohol vs. not-alcohol evaluation

Finally, we can take a look at how well the model performed at correctly classifying whether an image contains an alcoholic beverage (beer or wine). If no beverage was detected, we classify this as no alcohol present. There were no instances where the model incorrectly labelled not alcohol as alcohol!

In [ ]:
# Put the predictions and ground truth into the right format.
predicted_labels = ['alcohol' if label in {'beer', 'wine'} else 'not alcohol' for label in all_results]
ground_truth_labels = list(beverage_dataframe['alcohol_notalcohol'])

# Produce the metrics report.
generate_metric_report(ground_truth_labels, predicted_labels, label_names=['alcohol', 'not alcohol'])

# Generate the confusion matrix.
ConfusionMatrixDisplay.from_predictions(ground_truth_labels, predicted_labels, labels=['alcohol', 'not alcohol'], xticks_rotation='vertical', cmap='Blues', colorbar=False);

## Try Your Own Analysis

When running the following cell, you will be asked to select/input your data file using widgets that appear directly below the cell.

In [ ]:
try:
    # If we are on Google Colab, show an upload widget.
    from google.colab import files
    uploaded = files.upload()
    if uploaded:
        file_locations = list(uploaded.keys())
    else:
        file_locations = ''
except ModuleNotFoundError:
    # If we are not on Google Colab, ask for the names of all files.
    file_locations = []
    while True:
        next_location = input('Please enter the path to the image(s) you would like to classify. '
                              'Type \q to finish entering filepaths.')
        if next_location == '\q':
            break

for file_path in file_locations:
    if not os.path.isfile(file_path):
        print(f'File not found: {file_path}.')
        print('Please run this cell again.')

### CLIP

*Ensure that you have run through all previous code cells in the "Example 1" section first, as this code makes use of the classifier we initialised in that part of the tutorial.*

For simplicity, the code below does *not* use batching.

In [ ]:
# Here we specify the label options that the model will choose from.
# Make sure that you update these options to suit your data and experiment
# with different wordings.
candidate_labels = [
    'A photo containing beer',
    'A photo containing wine',
    'A photo containing water',
    'A photo containing coffee',
    'A photo containing tea',
    'A photo containing no drinks',
]

# Put into a dataframe for compatibility with the previous code.
custom_clip_dataframe = pd.DataFrame(file_locations, columns=['image_name'])

# Generate prediction results.
all_results = []
for index, sample in tqdm(custom_clip_dataframe.iterrows(), total=len(custom_clip_dataframe)):
    image = Image.open(sample['image_name'])
    inputs = clip_processor(text=candidate_labels, images=image, return_tensors='pt', padding=True).to(clip_device)
    with torch.no_grad():
        outputs = clip_model(**inputs)
    image_logits = outputs.logits_per_image[0].cpu()
    image_probs = image_logits.softmax(dim=0)
    all_results.append(image_probs)

# Save output to a CSV file.
save_clip_predictions('clip_predictions.csv', custom_clip_dataframe, all_results, candidate_labels)

### LLaVA

*Ensure that you have run through all previous code cells in the "Example 2" section first, as this code makes use of the classifier we initialised in that part of the tutorial.*

For simplicity, the code below does *not* use batching.

In [ ]:
# Here we specify the prompt for the model.
# Make sure that you update the prompt to suit your data and experiment with
# different wordings.
prompt = 'What type of beverage does this image contain? Pick the most relevant one from this list: beer, wine, coffee, tea, water, none. Answer in one word.'

# Here we specify how many tokens (words or word parts) can be returned by
# LLaVa. Increase this limit to enable the model to give a more detailed response,
# depending on the prompt.
max_new_tokens = 10

# Put into a dataframe for compatibility with the previous code.
custom_llava_dataframe = pd.DataFrame(file_locations, columns=['image_name'])

# Generate prediction results.
all_results = []
for index, sample in tqdm(custom_llava_dataframe.iterrows(), total=len(custom_llava_dataframe)):
    image = Image.open(sample['image_name'])
    llava_prompt = f'[INST] <image>\n{prompt} [/INST]'
    inputs = llava_processor(text=llava_prompt, images=image, return_tensors='pt').to(llava_device)
    with torch.no_grad():
        outputs = llava_model.generate(**inputs, max_new_tokens=max_new_tokens)
    generated_text = llava_processor.decode(outputs[0].cpu(), skip_special_tokens=True)
    response = generated_text.split(f'{prompt} [/INST]')[1].strip().lower()
    all_results.append(response)

# Save output to a CSV file.
save_llava_predictions('llava_predictions.csv', custom_llava_dataframe, all_results)